# Data Cleaning 1 
### Converting FemaWebDisasterSummaries.csv into DisasterSummaries.csv for use in our database

In [2]:
import pandas as pd

In [3]:
dirty_summary_table = pd.read_csv("FemaWebDisasterSummaries.csv")

In [4]:
dirty_summary_table.head()

,disasterNumber,totalNumberIaApproved,totalAmountIhpApproved,totalAmountHaApproved,totalAmountOnaApproved,totalObligatedAmountPa,totalObligatedAmountCatAb,totalObligatedAmountCatC2g,paLoadDate,iaLoadDate,hash,lastRefresh
0,1258,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4d19a1c0116ebbd291f446dd4f353f5d,2019-09-04T00:47:21.380Z
1,1268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5a9fb5ef3295e1e55f0901c821bb6e1f,2019-09-04T00:47:21.382Z
2,1241,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,777ead5c411cb2efb04436853d0b7443,2018-03-09T18:42:11.608Z
3,1265,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,b65172351ce90f9f3f1912f0cf401390,2019-09-04T00:47:21.380Z
4,1247,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,da36ec31dba54b471b394548e699d17a,2019-09-04T00:47:21.378Z


### Our PostgreSQL does not need data that references database maintenance.Create a new table that includes only the columns that we need. 

In [5]:
disasterSummaries = dirty_summary_table[
    ['disasterNumber', 'totalNumberIaApproved', 'totalAmountIhpApproved',
    'totalAmountHaApproved', 'totalAmountOnaApproved', 'totalObligatedAmountCatAb',
    'totalObligatedAmountCatC2g', 'paLoadDate', 'iaLoadDate']].copy()

In [6]:
disasterSummaries.head()

,disasterNumber,totalNumberIaApproved,totalAmountIhpApproved,totalAmountHaApproved,totalAmountOnaApproved,totalObligatedAmountCatAb,totalObligatedAmountCatC2g,paLoadDate,iaLoadDate
0,1258,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1241,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1265,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1247,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
#Check dtypes
print(disasterSummaries.dtypes)

disasterNumber                  int64
totalNumberIaApproved         float64
totalAmountIhpApproved        float64
totalAmountHaApproved         float64
totalAmountOnaApproved        float64
totalObligatedAmountCatAb     float64
totalObligatedAmountCatC2g    float64
paLoadDate                    float64
iaLoadDate                     object
dtype: object


### Remove hour:minute:second:timezone

In [9]:
disasterSummaries['iaLoadDate'] = disasterSummaries['iaLoadDate'].str.replace('T00:00:00.000Z', '')

In [30]:
disasterSummaries = disasterSummaries.rename(columns={'disasterNumber': 'disasternumber', 'totalNumberIaApproved': 'totalnumberiaapproved','totalAmountIhpApproved':'totalamountihpapproved', 'totalAmountHaApproved': 'totalamounthaapproved', 'totalAmountOnaApproved': 'totalamountonaapproved', 'totalObligatedAmountCatAb': 'totalobligatedamountcatab', 'totalObligatedAmountCatC2g':'totalobligatedamountcatc2g', 'paLoadDate': 'paloaddate', 'iaLoadDate': 'ialoaddate'})
print(disasterSummaries)

      disasternumber  totalnumberiaapproved  totalamountihpapproved  \
0               1258                    NaN                     NaN   
1               1268                    NaN                     NaN   
2               1241                    NaN                     NaN   
3               1265                    NaN                     NaN   
4               1247                    NaN                     NaN   
5               1272                    NaN                     NaN   
6               1243                    NaN                     NaN   
7               1239                    NaN                     NaN   
8               1278                    NaN                     NaN   
9               1250                    NaN                     NaN   
10              1275                    NaN                     NaN   
11              1257                    NaN                     NaN   
12              1285                    NaN                     NaN   
13    

# Export to PostgreSQL

In [24]:
from sqlalchemy import create_engine

In [25]:
conn = "postgres:Minnesota78!@localhost:5432/Project_2"
engine = create_engine((f'postgresql://{conn}'))

In [26]:
engine.table_names()

['disasterdeclarations', 'disastersummaries2', 'disastersummaries']

In [29]:
disasterSummaries.to_sql(name='disastersummaries', con=engine, if_exists='append', index=False)

ProgrammingError: (psycopg2.ProgrammingError) column "paLoadDate" of relation "disastersummaries" does not exist
LINE 1: ...obligatedamountcatab, totalobligatedamountcatc2g, "paLoadDat...
                                                             ^

[SQL: INSERT INTO disastersummaries (disasternumber, totalnumberiaapproved, totalamountihpapproved, totalamounthaapproved, totalamountonaapproved, totalobligatedamountcatab, totalobligatedamountcatc2g, "paLoadDate", "iaLoadDate") VALUES (%(disasternumber)s, %(totalnumberiaapproved)s, %(totalamountihpapproved)s, %(totalamounthaapproved)s, %(totalamountonaapproved)s, %(totalobligatedamountcatab)s, %(totalobligatedamountcatc2g)s, %(paLoadDate)s, %(iaLoadDate)s)]
[parameters: ({'disasternumber': 1258, 'totalnumberiaapproved': None, 'totalamountihpapproved': None, 'totalamounthaapproved': None, 'totalamountonaapproved': None, 'totalobligatedamountcatab': None, 'totalobligatedamountcatc2g': None, 'paLoadDate': None, 'iaLoadDate': None}, {'disas

In [28]:
pd.read_sql_query('select * from disastersummaries', con=engine).head()

,disasternumber,totalnumberiaapproved,totalamountihpapproved,totalamounthaapproved,totalamountonaapproved,totalobligatedamountpa,totalobligatedamountcatab,totalobligatedamountcatc2g,paloaddate,ialoaddate
